In [16]:
# Huggingface
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# Langchain
from langchain.llms import HuggingFacePipeline
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever, ContextualCompressionRetriever, ParentDocumentRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore

# Langchain Huggingface
from langchain_huggingface import HuggingFaceEmbeddings

# Langchain Community
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.vectorstores.utils import DistanceStrategy

# Langchain Core
from langchain_core.output_parsers import StrOutputParser

# Langchain Teddynote
from langchain_teddynote.retrievers import KiwiBM25Retriever

# Other
import torch
import os
import fitz
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from langchain_core.documents import Document

# Load PDF

In [3]:
##original code: https://dacon.io/competitions/official/236295/codeshare/11616?page=1&dtype=recent

pdfs = [
    "/home/yhkim/data/pdf_practice/test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf",
    "/home/yhkim/data/pdf_practice/test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf",
    "/home/yhkim/data/pdf_practice/test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf",
    "/home/yhkim/data/pdf_practice/test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf",
    "/home/yhkim/data/pdf_practice/test_source/국토교통부_행복주택출자.pdf",
    "/home/yhkim/data/pdf_practice/test_source/보건복지부_노인장기요양보험 사업운영.pdf",
    "/home/yhkim/data/pdf_practice/test_source/보건복지부_부모급여(영아수당) 지원.pdf",
    "/home/yhkim/data/pdf_practice/test_source/산업통상자원부_에너지바우처.pdf",
    "/home/yhkim/data/pdf_practice/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf"
]

doc_texts = []
for pdf_path in pdfs:
    doc = fitz.open(pdf_path)
    for page in doc:
        text = page.get_text()
    doc_texts.append(
        Document(
            page_content=text,
            metadata={"source": pdf_path},
        ))
        
        
        # {pdf_path: text})

In [25]:
doc_texts.append(Document(
            page_content="hello world",
            metadata={"source": "For examle"},
        ))

# Chunking

In [51]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", model_kwargs={"device": "cuda"})
dimension_size = len(embeddings.embed_query(""))

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [52]:
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

In [53]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=50
)
doc_texts = text_splitter.create_documents(texts=[i.page_content for i in doc_texts], metadatas=[i.metadata for i in doc_texts])


In [54]:
vectorstore = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [55]:
for i in doc_texts:
    await vectorstore.aadd_documents([i])

In [56]:
print(vectorstore.index.ntotal)
vectorstore.similarity_search_with_score("여러 pdf에서 결산사항들만 모아 보여줘", k=5)

47


[(Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/산업통상자원부_에너지바우처.pdf'}, page_content='비용 부담 완화를 위해 ‘22년 2차추경을 통해 91,617백만원을 증액하여 에너지바우처 지원\n대상을확대하고 지원단가를 인상\n2023\n- 예비비 편성 사유 : 취약계층 난방비 지원 특별대책(’23.1.26, 2.15.)에 따른 취약계층 에너지비용'),
  4.906433),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/국토교통부_행복주택출자.pdf'}, page_content='528,783\n- 건설규모 : ‘24년 2.2만호 준공 등 계속사업 추진\n②사업추진체계\n- 사업시행방법 : 출자, 보고\n- 사업시행주체 : 한국토지주택공사(LH), 지자체(지방공사)\n- 사업 수혜자 : 대\n학\n생\n・사\n회\n초\n년\n생・\n신혼부\n부 등\n 젊\n은층(80%), 고\n령자 및 주\n거취약\n계층\n(20%)'),
  5.00669),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/보건복지부_부모급여(영아수당) 지원.pdf'}, page_content='7. 사업 집행절차\n보건복지부\n지원기준 마련 및 국고보조금 교부\n↓\n시·도\n국고보조금에 지방비(시·도비)를 가산하여 시·군·구에 보조금 교부\n↓\n시·군·구\n지원대상자 선정, 급여액 확정 및 지급 및 정산\n※ e-지로시스템(금융결제원) 또는 각 시‧군‧구 금고 시스템을 통해 지급'),
  6.0614605),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'}, p

In [57]:
vectorstore = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
for i in doc_texts:
    await vectorstore.aadd_documents([i])
    
print(vectorstore.index.ntotal)
vectorstore.similarity_search_with_score("아이들의 지원금에 관한 문서를 찾아줘", k=10)

47


[(Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/산업통상자원부_에너지바우처.pdf'}, page_content='비용 부담 완화를 위해 ‘22년 2차추경을 통해 91,617백만원을 증액하여 에너지바우처 지원\n대상을확대하고 지원단가를 인상\n2023\n- 예비비 편성 사유 : 취약계층 난방비 지원 특별대책(’23.1.26, 2.15.)에 따른 취약계층 에너지비용'),
  3.482029),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/보건복지부_부모급여(영아수당) 지원.pdf'}, page_content='7. 사업 집행절차\n보건복지부\n지원기준 마련 및 국고보조금 교부\n↓\n시·도\n국고보조금에 지방비(시·도비)를 가산하여 시·군·구에 보조금 교부\n↓\n시·군·구\n지원대상자 선정, 급여액 확정 및 지급 및 정산\n※ e-지로시스템(금융결제원) 또는 각 시‧군‧구 금고 시스템을 통해 지급'),
  4.838092),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/국토교통부_행복주택출자.pdf'}, page_content='528,783\n- 건설규모 : ‘24년 2.2만호 준공 등 계속사업 추진\n②사업추진체계\n- 사업시행방법 : 출자, 보고\n- 사업시행주체 : 한국토지주택공사(LH), 지자체(지방공사)\n- 사업 수혜자 : 대\n학\n생\n・사\n회\n초\n년\n생・\n신혼부\n부 등\n 젊\n은층(80%), 고\n령자 및 주\n거취약\n계층\n(20%)'),
  4.9144745),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/산업통상자원부_에너지바우처.pdf'}, page_content='부담 완화를 위해

In [59]:
vectorstore = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatIP(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
for i in doc_texts:
    await vectorstore.aadd_documents([i])
    
print(vectorstore.index.ntotal)
vectorstore.similarity_search_with_score("아이들의 지원금에 관한 문서를 찾아줘", k=10)

47


[(Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'}, page_content='허등용.(2018). \x03\n보통교부세의 재정조정기능 분석과 재정분권정책에 주는 시사점. 한국지방세연구원 기본과\n제보고서 2018-06.\n홍근석·김봉균·임정빈.(2021). \x03\n우리나라의 정부 간 재정관계 분석: 자율성과 책임성을 중심으로. 한국정책과\n학학회보, 25(2): 147-173. \n홍근석·여효성(2021). \x03'),
  9.191544),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf'}, page_content='__. (2021). 2021회계연도 중앙관서별 재정사업 자율평가 보고서.\n박정수. (2020). dBrain 기반 재정성과관리 지원 강화 방안. 한국재정정보원 재정정보분석 20-03.\n박노욱 외. (2018). 재정성과평가제도 환류방안에 관한 연구. 한국조세재정연구원 연구보고서 18-14.'),
  8.838391),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf'}, page_content='_________. (2022). 「제11회 재정운용전략위원회 개최」.\n_________. (2022). 중기 재정사업 성과관리 추진방향 확정.\n_________. (2021). 「제2회 재정운용전략위원회 개최」.\n대한민국 정부. (2021). 국가재정법 일부개정법률안(대안), 2113611호.\n__________'),
  8.430627),
 (Document(metadata={'source

In [ ]:
[(Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/산업통상자원부_에너지바우처.pdf'}, page_content='비용 부담 완화를 위해 ‘22년 2차추경을 통해 91,617백만원을 증액하여 에너지바우처 지원\n대상을확대하고 지원단가를 인상\n2023\n- 예비비 편성 사유 : 취약계층 난방비 지원 특별대책(’23.1.26, 2.15.)에 따른 취약계층 에너지비용'),
  3.482029),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/보건복지부_부모급여(영아수당) 지원.pdf'}, page_content='7. 사업 집행절차\n보건복지부\n지원기준 마련 및 국고보조금 교부\n↓\n시·도\n국고보조금에 지방비(시·도비)를 가산하여 시·군·구에 보조금 교부\n↓\n시·군·구\n지원대상자 선정, 급여액 확정 및 지급 및 정산\n※ e-지로시스템(금융결제원) 또는 각 시‧군‧구 금고 시스템을 통해 지급'),
  4.838092),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/국토교통부_행복주택출자.pdf'}, page_content='528,783\n- 건설규모 : ‘24년 2.2만호 준공 등 계속사업 추진\n②사업추진체계\n- 사업시행방법 : 출자, 보고\n- 사업시행주체 : 한국토지주택공사(LH), 지자체(지방공사)\n- 사업 수혜자 : 대\n학\n생\n・사\n회\n초\n년\n생・\n신혼부\n부 등\n 젊\n은층(80%), 고\n령자 및 주\n거취약\n계층\n(20%)'),
  4.9144745),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/산업통상자원부_에너지바우처.pdf'}, page_content='부담 완화를 위해 예비비 100,000백만원 배정을 통한 동절기 지원단가 2배 인상(15.2→30.4만원) 지원\n□2023년 이․전용 등 세부내역 : 해당사항 없음\n□2023년 예비비 배정 세부내역\n(단위: 백만원)\n내역사업\n목\n세목\n금액\n예비비 배정 사유(날짜)\n동절기바우처\n민간경상보조\n민간경상보조\n133,033\n취약계층 난방비 지원'),
  5.580762),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'}, page_content='참고문헌\n국회예산정책처.(2022). 2022년 대한민국 지방재정. \n기획재정부.(2022). 2022년 국고보조사업 연장평가 보고서. \n김성주·윤태섭.(2019). \x03\n재정분권 추진에 따른 지방재정조정제도의 개선방안. 한국지방행정연구원 연구보고\n서 2019-09.\n김우림.(2021). 사회복지 분야 지방자치단체 국고보조사업 분석. 국회예산정책처.'),
  5.669753),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'}, page_content='박혜림.(2021). 지방교부세 법정률 인상 필요성 및 적정 법정률 도출. 한국지방세연구원 정책과제 2021-29. \n이재완 외.(2021). \x03\n중앙정부와 지방정부 간 재원분담 결정과정에 관한 탐색적 연구. 국회예산정책처 연구용\n역보고서.\n정종필·권진택.(2019). \x03\n보통교부세의 지역균형수요와 사회복지균형수요의 산정방법 및 효과분석. 한국지방'),
  6.3990993),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf'}, page_content='기획재정부. 제1회 재정운용전략위원회 보도자료. (2023.01.17.)\n기획재정부. 제2회 재정운용전략위원회 보도자료. (2023.06.14.)\n기획재정부. 2023년 국가재정전략회의 보도자료. (2023.06.28.)\n기획재정부. 제3회 재정운용전략위원회 보도자료. (2023.07.25.)\nGAO.\x03'),
  7.090667),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/보건복지부_노인장기요양보험 사업운영.pdf'}, page_content='○노인장기요양보험 운영지원\n○공무원·사립학교교원 등 장기요양보험료 국가부담금\n○기타의료급여 수급권자 급여비용 국가부담금\n○장기요양기관 재무회계 프로그램 운영\n부처\n피출연·피보조 기관\n보건복지부\n(972백만원)\n⇒\n(972백만원)\n한국사회보장정보원\n(972백만원)'),
  7.276239),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/국토교통부_행복주택출자.pdf'}, page_content='행복주택\n출자\n출자\nLH\n373,886\n30\n공\n공\n주\n택\n 특\n별\n법\n 제\n3조\n의\n2(공\n공\n주\n택\n의\n 재\n원\nㆍ\n세\n재\n지\n원\n 등\n)\n보자\n지자체\n154,897\n합계\n528,783'),
  7.3852854),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/국토교통부_행복주택출자.pdf'}, page_content="②추진경위\n- '13.04 서민 주거안정을 위한 주택시장 정상화 종합대책 발표\n(공공주택 연 13만호(국민임대 3.8만호 등 임대11만호, 분양2만호) 공급)\n* 행복주택은임대주택공급계획에포함('13∼'17년까지승인기준20만호)\n- '13.12 4.1, 8.28 부동산대책 후속조치 발표"),
  7.440755)]

In [61]:
db = FAISS.from_documents(doc_texts, embedding=embeddings)

db.similarity_search_with_score("Give me docs about stats", k=10)

[(Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/산업통상자원부_에너지바우처.pdf'}, page_content='동절기바우처\n민간경상보조\n민간경상보조\n133,033\n취약계층 난방비 지원 \n특별대책에 따른 동절기 \n지원단가 2배 인상 지원 \n(’23.2.23./10.25.)\n합  계\n133,033\n-\n라. 기타 추가자료 : 해당없음'),
  20.358929),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/보건복지부_노인장기요양보험 사업운영.pdf'}, page_content='내역사업명\n구분\n피보조․\n피출연 등 \n기관명\n지원 금액\n(2024예산)\n지원 \n비율(%)\n보조율 법적근거 (해당 조항)\n장기요양기관 \n재무회계\n프\n로\n그\n램\n 운\n영\n보조\n사회보장\n정보원\n972\n100%\n노인장기요양보험법 제35조의2\n7. 사업 집행절차\n○노인장기요양보험 운영지원\n○공무원·사립학교교원 등 장기요양보험료 국가부담금'),
  21.292845),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/국토교통부_행복주택출자.pdf'}, page_content='□주요내용 \n①사업규모\n- 사업기간 : ’13~계속\n- 최근 5년 간 투입된 사업비(예산액기준, 추경편성한 연도에는 추경포함)\n연도\n2020\n2021\n2022\n2023\n2024\n사업비\n1,267,123\n1,105,291\n775,293\n684,607\n528,783\n- 건설규모 : ‘24년 2.2만호 준공 등 계속사업 추진\n②사업추진체계'),
  21.349443),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test

In [62]:
db = FAISS.from_documents(doc_texts, embedding=embeddings, distance_strategy = DistanceStrategy.COSINE)

db.similarity_search_with_score("Give me docs about stats", k=10)

[(Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/산업통상자원부_에너지바우처.pdf'}, page_content='동절기바우처\n민간경상보조\n민간경상보조\n133,033\n취약계층 난방비 지원 \n특별대책에 따른 동절기 \n지원단가 2배 인상 지원 \n(’23.2.23./10.25.)\n합  계\n133,033\n-\n라. 기타 추가자료 : 해당없음'),
  20.358929),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/보건복지부_노인장기요양보험 사업운영.pdf'}, page_content='내역사업명\n구분\n피보조․\n피출연 등 \n기관명\n지원 금액\n(2024예산)\n지원 \n비율(%)\n보조율 법적근거 (해당 조항)\n장기요양기관 \n재무회계\n프\n로\n그\n램\n 운\n영\n보조\n사회보장\n정보원\n972\n100%\n노인장기요양보험법 제35조의2\n7. 사업 집행절차\n○노인장기요양보험 운영지원\n○공무원·사립학교교원 등 장기요양보험료 국가부담금'),
  21.292845),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/국토교통부_행복주택출자.pdf'}, page_content='□주요내용 \n①사업규모\n- 사업기간 : ’13~계속\n- 최근 5년 간 투입된 사업비(예산액기준, 추경편성한 연도에는 추경포함)\n연도\n2020\n2021\n2022\n2023\n2024\n사업비\n1,267,123\n1,105,291\n775,293\n684,607\n528,783\n- 건설규모 : ‘24년 2.2만호 준공 등 계속사업 추진\n②사업추진체계'),
  21.349443),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test